# extract netcdf4 grid data

In [2]:
import os, sys
from glob import glob 
import zipfile

In [3]:
fdir = 'F:\\vm\\AVHRR-LTDR\\grid\\v1.1'

In [23]:
zip_flist = glob(os.path.join(fdir, 'Compressed/*'))
zip_flist.sort()
print(zip_flist[1])
print(len(zip_flist))

F:\vm\AVHRR-LTDR\grid\v1.1\Compressed\1983.zip
37


In [11]:
extract_path = os.path.join(fdir, 'extract')
os.makedirs(extract_path, exist_ok=True)

In [ ]:
for zip_file_path in zip_flist:
    if zip_file_path.split('.')[-1].lower() != 'zip':  # skip if is not zip file
        continue
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

In [25]:
unzip_flist = glob(os.path.join(extract_path, '*'))
unzip_flist.sort()
print(len(unzip_flist))

432


In [32]:
# 检查是否每年的都解压出来了
year_dict = {}
for f in unzip_flist:
    date = os.path.basename(f).split('-')[0]
    year = date[:4]
    if year in year_dict.keys():
        year_dict[year].append(date)
    else:
        year_dict[year] = [date]
for year in year_dict.keys():
    if len(year_dict[year]) != 12:
        print(f'Issue found with year: {year}')
        break

# 设置 dimension，variable, metadata

In [1]:
import os, sys
from glob import glob 

import netCDF4 as nc
import numpy as np

In [2]:
fdir = 'F:\\vm\\AVHRR-LTDR\\grid\\v1.1'
extract_path = os.path.join(fdir, 'extract')
unzip_flist = glob(os.path.join(extract_path, '*'))
unzip_flist.sort()

In [16]:
def set_nc_attrs(nc_ds, config_file):
    with open(config_file, 'r') as fp:
        lines = fp.readlines()
    for line_raw in lines:
        line = line_raw.strip()
        attr_name = line.split(':')[0].strip()
        attr_val_raw = ''.join(line.split(':')[1:])
        attr_val = attr_val_raw.strip()
        nc_ds.setncattr(attr_name, attr_val)

In [30]:
# 读取变量定义并保存到文件中
with nc.Dataset(unzip_flist[0],'r') as ds:
    var_dict = ds.variables
    # no quantization info for this variable
    with open('./var_nc.txt','w') as fp:
        for var in var_dict.keys():
            fp.writelines(f'V==========V\n')
            fp.writelines(f'VAR:{var}\n')
            attr_keys = var_dict[var].ncattrs()
            for attr_key in attr_keys:
                attr_info = var_dict[var].getncattr(attr_key)
                fp.writelines(f'{attr_key}:{attr_info}\n')

In [10]:
with nc.Dataset(unzip_flist[0],'r') as ds:
    # print(dir(ds))
    # print(ds.title)
    # print(ds.ncattrs())
    # ds.title2 = 'shift'
    # var_dict = ds.variables
    # print(var_dict.keys())
    # print(var_dict['time'])
    # print(var_dict['lat'].dimensions)
    print(ds.dimensions.items())
    # dim_dict = ds.dimensions
    # print(dim_dict.keys())
    # print(dir(dim_dict['lat']))
    # print(dim_dict['lat'].name)
    # v = 'lon'
    # print(var_dict[v])
    # print(dir(var_dict[v]))

dict_items([('lat', <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 720), ('lon', <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 1440), ('nv', <class 'netCDF4._netCDF4.Dimension'>: name = 'nv', size = 2), ('strlen', <class 'netCDF4._netCDF4.Dimension'>: name = 'strlen', size = 150), ('time', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 1)])


In [7]:
os.getcwd()

'D:\\github\\avhrr_tools\\firecci'

In [7]:
# 设置attribute信息
# 没用上
def set_attr(ds_var, attr_name, attr_value):
    if attr_name in ds_var.ncattrs():
        ds_var.setncattr(attr_name, attr_value)
        return
    else:
        ds_var.NonameAttribute = attr_value
        ds_var.renameAttribute('NonameAttribute', attr_name)
    return

In [19]:
with nc.Dataset(unzip_flist[0],'r') as ds1:
    dim_dict = ds1.dimensions
    var_dict = ds1.variables
    with nc.Dataset('./test1.nc','w', format='NETCDF4') as ds2:
        # ?ds2.createDimension  # dimname, size=None)
        # ?ds2.createVariable  # varname, datatype, dimensions=()
        for dim_key in dim_dict.keys():
            dimv = dim_dict[dim_key]
            ds2.createDimension(dimname=dimv.name, size=dimv.size)
        for var_key in var_dict.keys():
            var_name = var_key
            var = var_dict[var_key]
            new_var = ds2.createVariable(
                varname=var_name, 
                datatype=var.datatype, 
                dimensions=var.dimensions
            )
            new_var.setncatts({k: var.getncattr(k) for k in var.ncattrs()})
            new_var[:] = var[:]
            
        ds2.title = 'combine file test'
        ds2.setncattr('author','Yuhua Situ') #和上一行命令所用的方法等价
        set_nc_attrs(ds2, './firecci_metadata.txt')

In [4]:
with open('./firecci_metadata.txt','r') as fp:
    lines = fp.readlines()
line = lines[1]

In [6]:
line.strip()

'institution: University of Alcala'

# 合并数据

In [1]:
import os, sys
from glob import glob 

import netCDF4 as nc
import numpy as np

In [2]:
fdir = 'F:\\vm\\AVHRR-LTDR\\grid\\v1.1'
extract_path = os.path.join(fdir, 'extract')
unzip_flist = glob(os.path.join(extract_path, '*'))
unzip_flist.sort()

In [3]:
print(unzip_flist[0])
print(unzip_flist[-1])

F:\vm\AVHRR-LTDR\grid\v1.1\extract\19820101-ESACCI-L4_FIRE-BA-AVHRR-LTDR-fv1.1.nc
F:\vm\AVHRR-LTDR\grid\v1.1\extract\20181201-ESACCI-L4_FIRE-BA-AVHRR-LTDR-fv1.1.nc


In [31]:
with nc.Dataset(fpath, 'r') as ds:
    print(ds.variables.keys())
    print(ds.variables['burned_area'].shape)
    print(ds.variables['standard_error'].shape)
    print(ds.variables['fraction_of_burnable_area'].shape)
    print(ds.variables['fraction_of_observed_area'].shape)
    # a = ds.variables['burned_area'][:]
    # b = np.moveaxis(a, [0,1,2], [2,1,0])
    # print(a.shape)
    # print(b.shape)

dict_keys(['lat', 'lat_bnds', 'lon', 'lon_bnds', 'time', 'time_bnds', 'burned_area', 'standard_error', 'fraction_of_burnable_area', 'fraction_of_observed_area'])
(1, 720, 1440)
(1, 720, 1440)
(1, 720, 1440)
(1, 720, 1440)


In [35]:
# 读取所有数据并且合并
fpath_base = 'F:\\vm\\AVHRR-LTDR\\grid\\v1.1\\extract\\{Ymd}-ESACCI-L4_FIRE-BA-AVHRR-LTDR-fv1.1.nc'
year = 1982
fpath_list = []

ind = 0
date_list = []
arr_burned_area = np.zeros( (1440, 720, len(unzip_flist)) )
arr_ste = np.zeros( (1440, 720, len(unzip_flist)) )
arr_burnable_frac = np.zeros( (1440, 720, len(unzip_flist)) )
arr_obs_frac = np.zeros( (1440, 720, len(unzip_flist)) )

coord_flag = True
while year < 2019:
    if year == 1994:
        year += 1
        continue
    print(f'working on year: {year}')
    for month in range(1, 13):
        fpath = fpath_base.replace('{Ymd}', f'{year}{month:02d}01')
        date_list.append(f'{year}{month:02d}01')
        # 读取 netcdf4, 1440, 720, 1
        # coords 'lat', 'lat_bnds', 'lon', 'lon_bnds'
        # 'burned_area', 'standard_error', 'fraction_of_burnable_area', 'fraction_of_observed_area'
        with nc.Dataset(fpath, 'r') as ds:
            arr_burned_area[:,:,ind] = np.moveaxis(ds.variables['burned_area'][0,:,:], [0,1], [1,0])
            arr_ste[:,:,ind] = np.moveaxis(ds.variables['standard_error'][0,:,:], [0,1], [1,0])
            arr_burnable_frac[:,:,ind] = np.moveaxis(ds.variables['fraction_of_burnable_area'][0,:,:], [0,1], [1,0])
            arr_obs_frac[:,:,ind]  = np.moveaxis(ds.variables['fraction_of_observed_area'][0,:,:], [0,1], [1,0])
            
            # 保存坐标信息，只需保存一次
            if coord_flag:
                coord_flag = False
                coord_dict = {}
                for key in ['lat','lat_bnds','lon','lon_bnds']:
                    coord_dict[key] = ds.variables[key][:]
        ind += 1
        fpath_list.append(fpath)
        
    year += 1

working on year: 1982
working on year: 1983
working on year: 1984
working on year: 1985
working on year: 1986
working on year: 1987
working on year: 1988
working on year: 1989
working on year: 1990
working on year: 1991
working on year: 1992
working on year: 1993
working on year: 1995
working on year: 1996
working on year: 1997
working on year: 1998
working on year: 1999
working on year: 2000
working on year: 2001
working on year: 2002
working on year: 2003
working on year: 2004
working on year: 2005
working on year: 2006
working on year: 2007
working on year: 2008
working on year: 2009
working on year: 2010
working on year: 2011
working on year: 2012
working on year: 2013
working on year: 2014
working on year: 2015
working on year: 2016
working on year: 2017
working on year: 2018


In [36]:
np.save(file='./coords.npy', arr=coord_dict, allow_pickle=True)

np.save(file='./burned_area.npy', arr={'date':date_list, 'arr':arr_burned_area}, 
        allow_pickle=True)
np.save(file='./ste.npy', arr={'date':date_list, 'arr':arr_ste}, 
        allow_pickle=True)
np.save(file='./frac_burnable.npy', arr={'date':date_list, 'arr':arr_burnable_frac}, 
        allow_pickle=True)
np.save(file='./frac_obs.npy', arr={'date':date_list, 'arr':arr_obs_frac}, 
        allow_pickle=True)


In [6]:
len(unzip_flist)

432

In [7]:
len(fpath_list)

432

In [19]:
with nc.Dataset(unzip_flist[0],'r') as ds1:
    dim_dict = ds1.dimensions
    print(dim_dict)
    print(ds1.variables.keys())
    print(ds1.variables['time_bnds'][:])
    print(ds1.variables['lat_bnds'][:])
    print(ds1.variables['lon_bnds'][:])
    print(ds1.variables['burned_area'])

{'lat': <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 720, 'lon': <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 1440, 'nv': <class 'netCDF4._netCDF4.Dimension'>: name = 'nv', size = 2, 'strlen': <class 'netCDF4._netCDF4.Dimension'>: name = 'strlen', size = 150, 'time': <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 1}
dict_keys(['lat', 'lat_bnds', 'lon', 'lon_bnds', 'time', 'time_bnds', 'burned_area', 'standard_error', 'fraction_of_burnable_area', 'fraction_of_observed_area'])
[[4383. 4413.]]
[[ 90.    89.75]
 [ 89.75  89.5 ]
 [ 89.5   89.25]
 ...
 [-89.25 -89.5 ]
 [-89.5  -89.75]
 [-89.75 -90.  ]]
[[-180.   -179.75]
 [-179.75 -179.5 ]
 [-179.5  -179.25]
 ...
 [ 179.25  179.5 ]
 [ 179.5   179.75]
 [ 179.75  180.  ]]
<class 'netCDF4._netCDF4.Variable'>
float32 burned_area(time, lat, lon)
    units: m2
    standard_name: burned_area
    long_name: total burned_area
    cell_methods: time: sum
unlimited dimensions: time
current shape = (1

In [ ]:
# 保存合并的数据到 netcdf 文件。暂不执行此操作。
with nc.Dataset(unzip_flist[0],'r') as ds1:
    dim_dict = ds1.dimensions
    var_dict = ds1.variables
    with nc.Dataset('./test1.nc','w', format='NETCDF4') as ds2:
        # ?ds2.createDimension  # dimname, size=None)
        # ?ds2.createVariable  # varname, datatype, dimensions=()
        for dim_key in dim_dict.keys():
            dimv = dim_dict[dim_key]
            ds2.createDimension(dimname=dimv.name, size=dimv.size)
        for var_key in var_dict.keys():
            var_name = var_key
            var = var_dict[var_key]
            new_var = ds2.createVariable(
                varname=var_name, 
                datatype=var.datatype, 
                dimensions=var.dimensions
            )
            new_var.setncatts({k: var.getncattr(k) for k in var.ncattrs()})
            new_var[:] = var[:]
            
        ds2.title = 'combine file test'
        ds2.setncattr('author','Yuhua Situ') #和上一行命令所用的方法等价
        set_nc_attrs(ds2, './firecci_metadata.txt')